In [7]:
import pandas as pd
import numpy as np
import os
import geopy.distance
import json
import requests

# from googlemaps import Client as GoogleMaps
from geopy.geocoders import Nominatim
from sklearn import preprocessing
from collections import namedtuple

## Constants

In [8]:
# Default API key: replace with your own api key
google_api_key = ""

data_path = "/Users/vincentchu/Dev/oakland-crime-prediction/data/"

## Functions

In [71]:
def float_catch_value_error(s):
    try:
        return float(s)
    except ValueError:
        return None
    
# Function to call Google's geocode api based on an address
# and returns a dictionary in json format
def get_geocode_json(address, api_key):
    geo_s ='https://maps.googleapis.com/maps/api/geocode/json'
    param = {'address': address, 'key': google_api_key }    
    response = requests.get(geo_s, params=param)
    result_dict = response.json()
    return result_dict

def convert_to_int(number_str):
    if number_str.find('(') > -1:
        return int(number_str[:number_str.find('(')])
    else:
        return int(number_str)

## Code

In [10]:
data = os.path.join(os.getcwd(),data_path + 'crimePublicData.csv')
crime_data = pd.read_csv(data)

In [11]:
print "Shape of crime_data: ", crime_data.shape
crime_data.describe()

Shape of crime_data:  (8927, 8)


,CrimeType,DateTime,CaseNumber,Description,PoliceBeat,Address,City,State
count,8927,8923,8927,8927,8885,8927,8927,8927
unique,14,6704,8913,235,62,4603,1,1
top,THEFT/LARCENY,2018-01-01 00:00:00,17-067802,BURGLARY-AUTO,04X,400 7TH ST,Oakland,CA
freq,2695,12,2,1485,477,101,8927,8927


In [12]:
crime_data["CrimeType"].unique()

array(['MOTOR VEHICLE THEFT', 'ROBBERY', 'VANDALISM', 'THEFT/LARCENY',
       'ASSAULT', 'HOMICIDE', 'FRAUD', 'WEAPONS',
       'DRUGS/ALCOHOL VIOLATIONS', 'BURGLARY', 'SEX CRIMES', 'DUI',
       'DISTURBING THE PEACE', 'ARSON'], dtype=object)

In [13]:
crime_data.columns

Index([u'CrimeType', u'DateTime', u'CaseNumber', u'Description', u'PoliceBeat',
       u'Address', u'City', u'State'],
      dtype='object')

In [14]:
crime_data["CrimeType"].value_counts()

THEFT/LARCENY               2695
ASSAULT                     1820
MOTOR VEHICLE THEFT         1407
VANDALISM                    970
BURGLARY                     534
ROBBERY                      523
DRUGS/ALCOHOL VIOLATIONS     303
DISTURBING THE PEACE         242
FRAUD                        112
WEAPONS                       93
SEX CRIMES                    87
DUI                           77
ARSON                         47
HOMICIDE                      17
Name: CrimeType, dtype: int64

In [15]:
crime_data.head()

,CrimeType,DateTime,CaseNumber,Description,PoliceBeat,Address,City,State
0,MOTOR VEHICLE THEFT,2104-08-08 00:00:00,14-040258,VEHICLE THEFT - AUTO,02X,12TH ST & MANDELA PKWY,Oakland,CA
1,ROBBERY,2030-01-11 19:00:00,14-023460,"ROBBERY - STRONG ARM (HANDS, FISTS, FEET, ETC.)",23X,3600 BLK OF INTERNATIONAL BL,Oakland,CA
2,VANDALISM,2040-02-01 21:40:00,14-035766,"VANDALISM [UNDER $1,000] (AMENDED)",09X,600 FAIRMOUNT AV,Oakland,CA
3,THEFT/LARCENY,2050-01-03 13:55:00,14-038177,BURGLARY-AUTO,08X,TELEGRAPH AV & W MACARTHUR BLVD,Oakland,CA
4,ASSAULT,2018-03-15 03:50:00,18-013298,FORCE/ADW-OTHER DANGEROUS WEAPON:GBI,24Y,35TH AV & SUTER ST,Oakland,CA


In [16]:
# Adding columns for Lat, Long and Zip
crime_data['Lat'] = 0.0
crime_data['Long'] = 0.0
crime_data['Zip'] = 0

crime_data_copy = crime_data.copy()

In [17]:
# Get Lat/Long and Zip for all addresses in the current set of crime data

for i in range(crime_data.shape[0]):
#for i in range(1001, 1005):    
    #print "Iteration i = " + str(i)
    
    address = crime_data.at[crime_data.index[i], 'Address'] + ", " + crime_data.at[crime_data.index[i], 'City'] + ", " + crime_data.at[crime_data.index[i], 'State']
    result_dict = get_geocode_json(address, google_api_key)

    if len(result_dict['results']) > 1:
        print "Iteration i = " + str(i) + ": There are " + str(len(result_dict['results'])) + " results for the following address:"
        print address
    
    try:
        crime_data.at[crime_data.index[i], 'Lat'] = result_dict['results'][0]['geometry']['location']['lat']
    except:
        print "i = " + str(i) + ": Cannot find Latitude"
    
    try:
        crime_data.at[crime_data.index[i], 'Long'] = result_dict['results'][0]['geometry']['location']['lng']
    except:
        print "i = " + str(i) + ": Cannot find Longitude"
        
    try:    
        for j in range(len(result_dict['results'][0]['address_components'])):
            if result_dict['results'][0]['address_components'][j]['types'][0] == 'postal_code':
                crime_data.at[crime_data.index[i], 'Zip'] = result_dict['results'][0]['address_components'][j]['long_name']
    except:
        print "i = " + str(i) + ": Cannot find Address Components"

Iteration i = 20: There are 2 results for the following address:
600 GRAND AV, Oakland, CA
Iteration i = 135: There are 2 results for the following address:
700 GRAND AV, Oakland, CA
Iteration i = 286: There are 2 results for the following address:
2400 VALLEY, Oakland, CA
Iteration i = 337: There are 2 results for the following address:
GRAND AV, Oakland, CA
Iteration i = 355: There are 10 results for the following address:
00 , Oakland, CA
i = 355: Cannot find Address Components
Iteration i = 515: There are 2 results for the following address:
300 ALICE ST, Oakland, CA
Iteration i = 521: There are 10 results for the following address:
00 , Oakland, CA
i = 521: Cannot find Address Components
Iteration i = 545: There are 10 results for the following address:
00 , Oakland, CA
i = 545: Cannot find Address Components
Iteration i = 622: There are 2 results for the following address:
10700 SKYLINE BLVD, Oakland, CA
Iteration i = 640: There are 10 results for the following address:
00 , Oakl

i = 6697: Cannot find Latitude
i = 6697: Cannot find Longitude
i = 6697: Cannot find Address Components
i = 6782: Cannot find Latitude
i = 6782: Cannot find Longitude
i = 6782: Cannot find Address Components
i = 7031: Cannot find Latitude
i = 7031: Cannot find Longitude
i = 7031: Cannot find Address Components
Iteration i = 7146: There are 10 results for the following address:
00 , Oakland, CA
i = 7146: Cannot find Address Components
Iteration i = 7180: There are 2 results for the following address:
5900 KEITH AV, Oakland, CA
Iteration i = 7269: There are 10 results for the following address:
00 , Oakland, CA
i = 7269: Cannot find Address Components
Iteration i = 7332: There are 10 results for the following address:
00 , Oakland, CA
i = 7332: Cannot find Address Components
Iteration i = 7345: There are 2 results for the following address:
27TH ST & NORTHGATE AV, Oakland, CA
Iteration i = 7530: There are 2 results for the following address:
GRAND AVE, Oakland, CA
Iteration i = 7578: The

In [20]:
# Create data frame with unique list of zip codes for Oakland
oakland_zip_codes = pd.DataFrame(crime_data.Zip.unique(), columns=['zip'])

In [21]:
# Get individual poverty data
data = os.path.join(os.getcwd(),data_path + 'ind_poverty_status_by_zip.csv')
ind_poverty_status = pd.read_csv(data)
ind_poverty_status = ind_poverty_status.drop(0, axis=0)
ind_poverty_status['zip'] = [int(ind_poverty_status.at[ind_poverty_status.index[i], 'GEO.id2']) for i in range(ind_poverty_status.shape[0])]
print "ind_poverty_status.shape:", ind_poverty_status.shape

# Filter out data not for Oakland
ind_poverty_status_oakland = ind_poverty_status.merge(oakland_zip_codes, 
                                                      left_on = 'zip', 
                                                      right_on = 'zip', 
                                                      how = 'inner')

print "ind_poverty_status_oakland.shape:", ind_poverty_status_oakland.shape
ind_poverty_status_oakland.head()

ind_poverty = ind_poverty_status_oakland[['zip', 'HC03_EST_VC01', 'HC03_MOE_VC01']]
ind_poverty

ind_poverty_status.shape: (1775, 370)
ind_poverty_status_oakland.shape: (37, 370)


,zip,HC03_EST_VC01,HC03_MOE_VC01
0,94025,7.8,1.3
1,94102,25.0,2.3
2,94123,5.1,1.0
3,94501,11.1,1.2
4,94505,6.0,2.0
5,94509,17.4,1.8
6,94526,3.4,1.1
7,94544,13.2,1.3
8,94551,6.3,1.4
9,94553,7.8,1.5


In [22]:
# Get family poverty data
data = os.path.join(os.getcwd(),data_path + 'family_poverty_status_by_zip.csv')
family_poverty_status = pd.read_csv(data)
family_poverty_status = family_poverty_status.drop(0, axis=0)
family_poverty_status['zip'] = [int(family_poverty_status.at[ind_poverty_status.index[i], 'GEO.id2']) for i in range(ind_poverty_status.shape[0])]
print "family_poverty_status.shape:", family_poverty_status.shape

# Filter out data not for Oakland
family_poverty_status_oakland = family_poverty_status.merge(oakland_zip_codes, 
                                                            left_on = 'zip', 
                                                            right_on = 'zip', 
                                                            how = 'inner')

print "family_poverty_status_oakland.shape:", family_poverty_status_oakland.shape
family_poverty_status_oakland.head()

family_poverty = family_poverty_status_oakland[['zip', 'HC02_EST_VC01', 'HC02_MOE_VC01']]
family_poverty.head(10)

family_poverty_status.shape: (1775, 604)
family_poverty_status_oakland.shape: (37, 604)


/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,3,4,7,8,11,12,15,16,19,20,23,24,27,28,31,32,35,36,39,40,43,44,47,48,51,52,55,56,59,60,63,64,67,68,71,72,75,76,79,80,83,84,87,88,91,92,95,96,99,100,103,104,107,108,111,112,115,116,119,120,123,124,127,128,131,132,135,136,139,140,143,144,147,148,151,152,155,156,159,160,163,164,167,168,171,172,175,176,179,180,183,184,187,188,191,192,195,196,199,200,203,204,207,208,211,212,215,216,219,220,223,224,227,228,231,232,235,236,239,240,243,244,247,248,251,252,255,256,259,260,263,264,267,268,271,272,275,276,279,280,283,284,287,288,291,292,295,296,299,300,303,304,307,308,311,312,315,316,319,320,323,324,327,328,331,332,335,336,339,340,343,344,347,348,351,352,355,356,359,360,363,364,367,368,371,372,375,376,379,380,383,384,387,388,391,392,395,396,399,400,403,404,407,408,411,412,415,416,419,420,423,424,427,428,431,432,435,436,439,440,443,444,447,448,451,452,455,456,459,460,463,464,467,468,483,484,487,48

,zip,HC02_EST_VC01,HC02_MOE_VC01
0,94025,4.8,1.3
1,94102,11.9,3.5
2,94123,2.8,1.4
3,94501,8.1,1.2
4,94505,4.3,2.0
5,94509,14.2,1.8
6,94526,2.0,0.9
7,94544,10.9,1.4
8,94551,4.1,1.4
9,94553,5.8,1.6


In [23]:
poverty = ind_poverty.merge(family_poverty, how = 'inner', left_on = 'zip', right_on = 'zip')
print "poverty.shape: ", poverty.shape

poverty.columns = ['zip', 'IndPovertyPct', 'IndPovertyMoe', 'FamPovertyPct', 'FamPovertyMoe']

poverty['IndPovertyPct'] = [float_catch_value_error(poverty.at[poverty.index[i], 'IndPovertyPct']) for i in range(poverty.shape[0])]
poverty['IndPovertyMoe'] = [float_catch_value_error(poverty.at[poverty.index[i], 'IndPovertyMoe']) for i in range(poverty.shape[0])]
poverty['FamPovertyPct'] = [float_catch_value_error(poverty.at[poverty.index[i], 'FamPovertyPct']) for i in range(poverty.shape[0])]
poverty['FamPovertyMoe'] = [float_catch_value_error(poverty.at[poverty.index[i], 'FamPovertyMoe']) for i in range(poverty.shape[0])]

poverty.tail(12)

poverty.shape:  (37, 5)


,zip,IndPovertyPct,IndPovertyMoe,FamPovertyPct,FamPovertyMoe
25,94613,40.0,60.0,NaN,NaN
26,94618,4.7,1.4,1.4,1.2
27,94619,10.4,2.5,8.9,2.8
28,94621,34.6,3.3,33.2,3.5
29,94702,15.4,2.7,7.6,3.1
30,94703,16.0,2.6,7.2,3.4
31,94704,50.0,3.6,13.6,6.2
32,94705,10.3,2.1,4.3,2.1
33,94708,4.3,1.3,0.8,0.8
34,94720,45.7,54.3,55.2,55.2


In [24]:
# Merge crime_data with poverty data

crime_data_v2 = crime_data.merge(poverty, left_on = "Zip", right_on = 'zip')
crime_data_v2 = crime_data_v2.drop('zip', 1)
crime_data_v2.tail(10)

,CrimeType,DateTime,CaseNumber,Description,PoliceBeat,Address,City,State,Lat,Long,Zip,IndPovertyPct,IndPovertyMoe,FamPovertyPct,FamPovertyMoe
8739,THEFT/LARCENY,2018-01-29 21:00:00,18-008988,THEFT,99X,3300 EDINBURGH RD,Oakland,CA,37.692529,-121.761092,94551,6.3,1.4,4.1,1.4
8740,HOMICIDE,2018-01-01 17:22:00,18-000116,MURDER,07X,3400 ADELINE ST,Oakland,CA,37.846578,-122.273032,94703,16.0,2.6,7.2,3.4
8741,MOTOR VEHICLE THEFT,2018-01-03 17:40:00,18-000525,VEHICLE THEFT - AUTO,77X,4300 GRIZZLY PEAK BLVD,Oakland,CA,37.868139,-122.216114,94563,2.6,1.1,1.9,1.1
8742,THEFT/LARCENY,2017-12-28 22:15:00,17-067667,BURGLARY-AUTO,07X,1500 BEACH ST,Oakland,CA,37.804954,-122.433725,94123,5.1,1.0,2.8,1.4
8743,THEFT/LARCENY,2018-03-13 11:21:00,18-012965,THEFT,99X,600 ESCOBAR ST,Oakland,CA,38.017387,-122.137973,94553,7.8,1.5,5.8,1.6
8744,VANDALISM,2018-02-18 03:37:00,18-902987,VANDALISM,NaN,100 E ST,Oakland,CA,38.002603,-122.133408,94553,7.8,1.5,5.8,1.6
8745,THEFT/LARCENY,2018-03-13 11:21:00,18-012966,PETTY THEFT,99X,900 CHANSLOR AVE,Oakland,CA,37.932475,-122.360912,94801,24.7,3.3,23.3,3.0
8746,VANDALISM,2018-02-16 22:10:00,18-902956,VANDALISM,NaN,GRAND & LAKESHORE,Oakland,CA,37.911775,-121.619089,94505,6.0,2.0,4.3,2.0
8747,THEFT/LARCENY,2018-03-08 07:20:00,18-012079,RECEIVE/ETC KNOWN STOLEN PROPERTY,99X,500 CABANA WAY,Oakland,CA,37.783415,-121.965128,94526,3.4,1.1,2.0,0.9
8748,ASSAULT,2018-03-06 02:42:00,18-011607,BATTERY:SPOUSE/EX SPOUSE/DATE/ETC,77X,2400 DURANT AV,Oakland,CA,37.867385,-122.260958,94720,45.7,54.3,55.2,55.2


In [25]:
# write crime_data to csv
#crime_data.to_csv('crimePublicData_transformed.csv')
crime_data_v2.to_csv('crimePublicData_transformed_v2.csv')

# Examine head of the data frame for validation
crime_data_v2.head()

,CrimeType,DateTime,CaseNumber,Description,PoliceBeat,Address,City,State,Lat,Long,Zip,IndPovertyPct,IndPovertyMoe,FamPovertyPct,FamPovertyMoe
0,MOTOR VEHICLE THEFT,2104-08-08 00:00:00,14-040258,VEHICLE THEFT - AUTO,02X,12TH ST & MANDELA PKWY,Oakland,CA,37.809085,-122.291942,94607,29.9,2.8,27.7,3.3
1,THEFT/LARCENY,2018-01-17 13:05:00,18-006602,GRAND THEFT:MONEY/LABOR/PROPERTY OVER $400,03Y,400 7TH ST,Oakland,CA,37.799049,-122.272988,94607,29.9,2.8,27.7,3.3
2,ASSAULT,2018-03-15 16:10:00,18-013433,BATTERY:SPOUSE/EX SPOUSE/DATE/ETC,02Y,700 PERALTA ST,Oakland,CA,37.806311,-122.297974,94607,29.9,2.8,27.7,3.3
3,DUI,2018-01-17 14:00:00,18-006608,DUI ALCOHOL/DRUGS,03Y,400 7TH ST,Oakland,CA,37.799049,-122.272988,94607,29.9,2.8,27.7,3.3
4,ASSAULT,2018-03-15 19:25:00,18-013466,BATTERY,03X,800 FRANKLIN ST,Oakland,CA,37.799789,-122.272333,94607,29.9,2.8,27.7,3.3


## Adding housing data

In [29]:
#data_path_ml = "/Users/vincentchu/Dev/oakland-crime-prediction/machine_learning/machine_learning/"

data = os.path.join(os.getcwd(), data_path + 'poverty_housing_oakland.csv')
housing = pd.read_csv(data)

In [69]:
housing.columns

Index([u'Unnamed: 0', u'zip', u'tract',
       u'Number; HOUSING OCCUPANCY - Total housing units',
       u'Percent; HOUSING OCCUPANCY - Total housing units',
       u'Number; HOUSING OCCUPANCY - Total housing units - Occupied housing units',
       u'Percent; HOUSING OCCUPANCY - Total housing units - Occupied housing units',
       u'Number; HOUSING OCCUPANCY - Total housing units - Vacant housing units',
       u'Percent; HOUSING OCCUPANCY - Total housing units - Vacant housing units',
       u'Number; HOUSING OCCUPANCY - Total housing units - Vacant housing units - For rent',
       u'Percent; HOUSING OCCUPANCY - Total housing units - Vacant housing units - For rent',
       u'Number; HOUSING OCCUPANCY - Total housing units - Vacant housing units - Rented, not occupied',
       u'Percent; HOUSING OCCUPANCY - Total housing units - Vacant housing units - Rented, not occupied',
       u'Number; HOUSING OCCUPANCY - Total housing units - Vacant housing units - For sale only',
       u'Pe

In [70]:
housing_v2 = housing.copy()

housing_v2['total_units'] = map(convert_to_int, housing_v2['Number; HOUSING OCCUPANCY - Total housing units'])
housing_v2['occupied_units'] = map(convert_to_int, housing_v2['Number; HOUSING OCCUPANCY - Total housing units - Occupied housing units'])
housing_v2['vacant_units'] = map(convert_to_int, housing_v2['Number; HOUSING OCCUPANCY - Total housing units - Vacant housing units'])

housing_v2['occupied_pct'] = housing_v2['occupied_units'] * 1.0 / housing_v2['total_units']
housing_v2['vacant_pct'] = housing_v2['vacant_units'] * 1.0 / housing_v2['total_units']

housing_v2 = housing_v2[['zip', 'total_units', 'occupied_units', 'vacant_units', 'occupied_pct', 'vacant_pct']]
housing_v2.head(10)

,zip,total_units,occupied_units,vacant_units,occupied_pct,vacant_pct
0,94025,1605,1504,101,0.937072,0.062928
1,94025,1657,1601,56,0.966204,0.033796
2,94025,1461,1384,77,0.947296,0.052704
3,94025,919,866,53,0.942329,0.057671
4,94025,1515,1372,143,0.905611,0.094389
5,94025,2291,2158,133,0.941947,0.058053
6,94025,1192,1157,35,0.970638,0.029362
7,94025,843,816,27,0.967972,0.032028
8,94025,1566,1517,49,0.968710,0.031290
9,94025,826,787,39,0.952785,0.047215


In [83]:
crime_data_v3 = crime_data_v2.copy()

crime_data_v3 = crime_data_v3.merge(housing_v2[['zip', 'occupied_pct', 'vacant_pct']], 
                                    left_on = 'Zip', 
                                    right_on = 'zip', 
                                    how = 'left')

crime_data_v3 = crime_data_v3.drop(['zip'], axis = 1)

crime_data_v3.head()

,CrimeType,DateTime,CaseNumber,Description,PoliceBeat,Address,City,State,Lat,Long,Zip,IndPovertyPct,IndPovertyMoe,FamPovertyPct,FamPovertyMoe,occupied_pct,vacant_pct
0,MOTOR VEHICLE THEFT,2104-08-08 00:00:00,14-040258,VEHICLE THEFT - AUTO,02X,12TH ST & MANDELA PKWY,Oakland,CA,37.809085,-122.291942,94607,29.9,2.8,27.7,3.3,0.883426,0.116574
1,MOTOR VEHICLE THEFT,2104-08-08 00:00:00,14-040258,VEHICLE THEFT - AUTO,02X,12TH ST & MANDELA PKWY,Oakland,CA,37.809085,-122.291942,94607,29.9,2.8,27.7,3.3,0.942073,0.057927
2,MOTOR VEHICLE THEFT,2104-08-08 00:00:00,14-040258,VEHICLE THEFT - AUTO,02X,12TH ST & MANDELA PKWY,Oakland,CA,37.809085,-122.291942,94607,29.9,2.8,27.7,3.3,0.791383,0.208617
3,MOTOR VEHICLE THEFT,2104-08-08 00:00:00,14-040258,VEHICLE THEFT - AUTO,02X,12TH ST & MANDELA PKWY,Oakland,CA,37.809085,-122.291942,94607,29.9,2.8,27.7,3.3,0.832427,0.167573
4,MOTOR VEHICLE THEFT,2104-08-08 00:00:00,14-040258,VEHICLE THEFT - AUTO,02X,12TH ST & MANDELA PKWY,Oakland,CA,37.809085,-122.291942,94607,29.9,2.8,27.7,3.3,0.866051,0.133949


In [84]:
# write crime_data to csv
crime_data_v3.to_csv('crimePublicData_transformed_v3.csv')

## Testing Section

In [73]:
crime_data_v2.query("Zip != 94607").head()

,CrimeType,DateTime,CaseNumber,Description,PoliceBeat,Address,City,State,Lat,Long,Zip,IndPovertyPct,IndPovertyMoe,FamPovertyPct,FamPovertyMoe
1063,ROBBERY,2030-01-11 19:00:00,14-023460,"ROBBERY - STRONG ARM (HANDS, FISTS, FEET, ETC.)",23X,3600 BLK OF INTERNATIONAL BL,Oakland,CA,37.775883,-122.221834,94601,27.5,2.3,25.0,2.4
1064,WEAPONS,2018-03-15 13:15:00,18-013400,PROHIBITED POSSESS AMMUNITION OR RELOADED AMMU...,27X,1400 44TH AV,Oakland,CA,37.771996,-122.213942,94601,27.5,2.3,25.0,2.4
1065,MOTOR VEHICLE THEFT,2018-03-15 15:25:00,18-013423,VEHICLE THEFT - AUTO,21Y,3200 LYNDE ST,Oakland,CA,37.789930,-122.216164,94601,27.5,2.3,25.0,2.4
1066,DRUGS/ALCOHOL VIOLATIONS,2018-03-15 17:45:00,18-013450,POSSESS CONTROLLED SUBSTANCE PARAPHERNALIA,21X,2100 MITCHELL ST,Oakland,CA,37.785326,-122.226034,94601,27.5,2.3,25.0,2.4
1067,VANDALISM,2040-01-01 00:00:00,14-031456,"VANDALISM [UNDER $1,000] (AMENDED)",20X,2600 E 12TH ST,Oakland,CA,37.780145,-122.232341,94601,27.5,2.3,25.0,2.4


In [ ]:
# Print out Lat/Long and Zip of part of the data frame for validation
for i in range(101, 105):
    print "Iteration %d" % i
    print "(Lat, Long) = ", (crime_data_v2.at[crime_data.index[i], 'Lat'], crime_data_v2.at[crime_data.index[i], 'Long'])
    print "Zip = %d" % crime_data_v2.at[crime_data.index[i], 'Zip']
    print "% Individuals below poverty line = ", crime_data_v2.at[crime_data.index[i], 'IndPovertyPct'], "%"
    print "% Families below poverty line = ", crime_data_v2.at[crime_data.index[i], 'FamPovertyPct'], "%"

In [ ]:
data_path = "/Users/vincentchu/Dev/oakland-crime-prediction/data/"

data = os.path.join(os.getcwd(),data_path + 'crimePublicData_transformed_v2.csv')
transformed_crime_data_v2 = pd.read_csv(data)

In [ ]:
transformed_crime_data_v2.columns

In [ ]:
transformed_crime_data_v2.Zip.unique()

In [ ]:
transformed_crime_data_v2.head(10)